In [50]:
from spacy.lang.en import English
from common import create_tops_patterns
import spacy

In [ ]:
create_tops_patterns()

In [40]:
'''
#old_version spaCy 2.0 and not usable anymore
nlp = English()
ruler = EntityRuler(nlp)
ruler.add_patterns([{'label':'TOPS', 'pattern': p} for p in create_tops_patterns()])
nlp.add_pipe(ruler)
'''


In [63]:
#initilize empty pipeline model
#nlp = English()

nlp = spacy.load("en_core_web_sm")
#intilize EntityRuler
entity_ruler_TOPS = nlp.add_pipe("entity_ruler")
entity_ruler_TOPS.add_patterns([{'label':'TOPS', 'pattern': p} for p in create_tops_patterns()])

In [64]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1509ed820>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1509ed8e0>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x157d30890>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x17eb2b240>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x17eb41840>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x157d30eb0>),
 ('entity_ruler', <spacy.pipeline.entityruler.EntityRuler at 0x151b4d940>)]

In [77]:
doc = nlp("Luis Lin like my U.S. stylish sweater, camisole and T-shirt. My beatiful sweater is stylish in China")
print([(ent.text, ent.label_) for ent in doc.ents])

[('Luis Lin', 'PERSON'), ('U.S.', 'GPE'), ('sweater', 'TOPS'), ('camisole', 'TOPS'), ('T-shirt', 'TOPS'), ('sweater', 'TOPS'), ('China', 'GPE')]


In [78]:
from spacy import displacy
displacy.render(doc, style='ent')

Save the entity_ruler for 'TOPS' to disk

In [53]:
entity_ruler_TOPS.to_disk("enetity_ruler_TOPS.jsonl")
nlp.to_disk("rule-model-TOPS")

Preparing Data for Scoring
Need some data to compare both method.(certain format)

In [58]:
from spacy.training import docs_to_json
from spacy.training.iob_utils import biluo_tags_from_offsets, spans_from_biluo_tags
import srsly


In [60]:
TRAIN_DATA  = [('These camisole and T-shirt and are so good. I did have similar pattern jacket which is so fancy', {'entities': [(6, 14, 'TOPS'), (19, 26, 'TOPS'), (71, 77, 'TOPS')]})]


In [61]:
nlp = spacy.load('en_core_web_sm')
docs = []
for text, annot in TRAIN_DATA:
    doc = nlp(text)
    tags = biluo_tags_from_offsets(doc, annot['entities'])
    entities = spans_from_biluo_tags(doc, tags)
    doc.ents = entities
    docs.append(doc)

srsly.write_json("spacy_format.json", [docs_to_json(docs)])